In [6]:
# 设置OpenAI API密钥
import os

from torch.backends.mkl import verbose

os.environ["DASHSCOPE_API_KEY"] = 'sk-API_KEY'

# 导入所需的库和模块
from langchain.schema import (
    HumanMessage,
    SystemMessage
)
from langchain_community.chat_models import ChatTongyi
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chains.llm import LLMChain


In [4]:
class CommandlineChatBot:
    def __init__(self):
        self.chat = ChatTongyi(model='qwen-max', temperature=0)
        self.messages = [SystemMessage(content="你是一个花卉行家。")]

    def chat_loop(self):
        print("ChatBot已启动！exit会退出")
        while True:
            user_input = input("> ")
            if user_input.lower() == "exit":
                print("exit")
                break
            self.messages.append(HumanMessage(content=user_input))
            response = self.chat.invoke(self.messages)
            print(f"ChatBot: {response.content}")


if __name__ == '__main__':
    bot = CommandlineChatBot()
    bot.chat_loop()

ChatBot已启动！exit会退出
ChatBot: 你好！作为你的花卉顾问，我在这里帮助你解决关于花卉种植、养护、识别或任何与花卉相关的问题。无论你是初学者还是经验丰富的园艺爱好者，都可以随时向我提问。想了解什么花适合你的花园，如何照顾某种特定的植物，或是解决花卉生长中遇到的问题，尽管告诉我吧！
ChatBot: 你好！红色花朵通常象征着热情、爱情和欲望，非常适合用来表达对恋人的深厚情感。给喜欢红色的女友送花，以下几个选项都是不错的选择：

1. **红玫瑰**：这是最经典的爱情之花，代表着热烈的爱情和美丽。如果想要表达浓烈的爱意，红玫瑰是不二之选。

2. **红百合**：象征着高贵、纯洁和深深的爱。相比玫瑰，百合多了一份清新与高雅，适合气质温婉的女性。

3. **红康乃馨**：代表着真爱、敬重与温馨的祝福。康乃馨花语中蕴含着对对方深深的关怀和不变的情感。

4. **红郁金香**：在花语中代表着“热烈的爱意”，它比玫瑰更显独特和浪漫，适合用来表达深情且特别的情感。

5. **红绣球花**：绣球花整体饱满，红色绣球花代表着美满团圆和忠贞不渝的爱情，适合用来表达希望与对方共度未来的愿望。

选择时，你可以考虑她的个性和喜好，以及你们的关系阶段来决定最合适的花种。如果想让礼物更加个性化，可以在花束中加入一些她特别喜欢的小配饰或手写一封情书，这样的细节会让礼物更加贴心和难忘。
exit


In [14]:
class ChatBotWithMemory:
    def __init__(self):
        self.llm = ChatTongyi(model='qwen-max', temperature=0)

        # 初始化 Prompts
        self.prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(
                    "你是一个花卉行家，你通常回答不会超过50个字。"
                ),
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("{question}")
            ]
        )

        # 初始化Memory
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            verbose=True,
            memory=self.memory
        )

    def chat_loop(self):
        print("Chatbot 已启动！")
        while True:
            user_input = input("> ")
            if user_input.lower() == "exit":
                print("exit")
                break

            response = self.conversation.invoke({"question": user_input})
            print(f"Chatbot: {response['text']}")


if __name__ == '__main__':
    bot = ChatBotWithMemory()
    bot.chat_loop()


Chatbot 已启动！


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: 你是一个花卉行家，你通常回答不会超过50个字。
Human: 你好，我想给我女朋友送一束花，什么好呢？

> Finished chain.
Chatbot: 玫瑰代表爱情，经典选择；百合清雅，象征纯洁；郁金香高贵，表达爱意。考虑她的喜好和花语，任意选都心意满满。


KeyboardInterrupt: Interrupted by user

In [19]:

# 导入所需的库
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import DashScopeEmbeddings

# ChatBot类的实现-带检索功能
class ChatbotWithRetrieval:

    def __init__(self, dir):

        # 加载Documents
        base_dir = dir  # 文档的存放目录
        documents = []
        for file in os.listdir(base_dir):
            file_path = os.path.join(base_dir, file)
            if file.endswith('.pdf'):
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
            elif file.endswith('.docx') or file.endswith('.doc'):
                loader = Docx2txtLoader(file_path)
                documents.extend(loader.load())
            elif file.endswith('.txt'):
                loader = TextLoader(file_path)
                documents.extend(loader.load())

        # 文本的分割
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
        all_splits = text_splitter.split_documents(documents)

        # 向量数据库
        self.vectorstore = Qdrant.from_documents(
            documents=all_splits,  # 以分块的文档
            embedding=DashScopeEmbeddings(),  # 用OpenAI的Embedding Model做嵌入
            location=":memory:",  # in-memory 存储
            collection_name="my_documents", )  # 指定collection_name

        # 初始化LLM
        self.llm = ChatTongyi(model='qwen-max', temperature=0)

        # 初始化Memory
        self.memory = ConversationSummaryMemory(
            llm=self.llm,
            memory_key="chat_history",
            return_messages=True
        )

        # 设置Retrieval Chain
        retriever = self.vectorstore.as_retriever()
        self.qa = ConversationalRetrievalChain.from_llm(
            self.llm,
            retriever=retriever,
            memory=self.memory
        )

    # 交互对话的函数
    def chat_loop(self):
        print("Chatbot 已启动! 输入'exit'来退出程序。")
        while True:
            user_input = input("你: ")
            if user_input.lower() == 'exit':
                print("再见!")
                break
            # 调用 Retrieval Chain  
            response = self.qa(user_input)
            print(f"Chatbot: {response['answer']}")


if __name__ == "__main__":
    # 启动Chatbot
    folder = "./"
    bot = ChatbotWithRetrieval(folder)
    bot.chat_loop()


Chatbot 已启动! 输入'exit'来退出程序。
Chatbot: 当然，我可以告诉你一些花的寓意：

1. **牡丹**：繁荣、财富、幸福
2. **向日葵**：忠诚、阳光、积极
3. **满天星**（Gypsophila）：纯洁、浪漫、美丽的梦想
4. **桃花**：浪漫的运气、美好的命运、幸福
5. **绣球花**：真诚、感激、最好的祝愿
6. **蝴蝶兰**（Phalaenopsis）：高贵、优雅、美丽动人
7. **樱花**：美丽、短暂、美好时刻
8. **铃兰**：幸福、纯洁、深情的爱
9. **玫瑰**：爱、热情、美丽
10. **康乃馨**：母爱、纯真、尊敬
11. **百合**：纯洁、高贵、祝福
12. **郁金香**：完美的爱、激情、温柔
13. **雏菊**：纯真、美丽、希望
14. **薰衣草**：宁静、纯真、追求爱情
15. **狐狸手套**（Foxglove）：美丽、喜悦、祝福
16. **石竹花**（Dianthus）：细腻的思绪、真诚、浪漫
17. **木兰**：高贵、优雅、永恒的爱

每种花都有其独特的意义，适合在不同场合表达不同的情感。
Chatbot: 易速鲜花网站是一个服务型的平台，它不仅提供在线购花服务，还融入了丰富的文化、美食、音乐等元素，旨在创造独特而全面的网上购花体验。该网站超越了传统网络鲜花运营商的经营模式，结合了乐、吃、住、行、游等多个方面，构建了一个展现中国原生态特色的网上鲜花服务平台。通过深度挖掘鲜花的情感价值，易速鲜花还提供文化展示服务，如酒文化、茶文化、农耕文化等，让购花成为一次综合的文化之旅。
Chatbot: 易速鲜花网站成功的关键因素在于它不仅仅是一个售卖鲜花的平台，而是一个集服务、体验、文化和综合经济促进为一体的创新概念。具体来说：

1. **高品位服务与经营理念**：易速鲜花提供高品位服务，结合超前的经营理念，创造了独特的网上购花体验。
2. **综合服务体验**：购花被赋予文化、美食、音乐等多重体验，使其超越单纯的商品交易，成为一次全方位的享受。
3. **经营模式创新**：它超越了传统网络鲜花运营商的单一模式，通过融入多种元素（如乐、吃、住、行、游），打造特色服务平台。
4. **文化与情感内涵**：每朵鲜花被赋予深厚的情感和文化价值，通过展示酒文化、茶文化、农耕文化等，增强用户体验的

KeyboardInterrupt: Interrupted by user